In [ ]:
!pip install transformers trl datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from trl import RewardTrainer, RewardConfig
from datasets import Dataset
import pandas as pd

# Step 1: Prepare the dataset
raw_data = [
    {
        "prompt": "Tell me a joke.",
        "response_1": "Why did the chicken cross the road?",
        "response_2": "Knock, knock!",
        "label": 1,  # 1 means response_1 is preferred
    },
    {
        "prompt": "What is AI?",
        "response_1": "Artificial Intelligence.",
        "response_2": "A type of plant.",
        "label": 0,  # 0 means response_2 is preferred
    },
]

# Convert raw data into a DataFrame and preprocess it
df = pd.DataFrame(raw_data)

# Create a dataset with "chosen" and "rejected" columns
def map_responses(row):
    if row["label"] == 1:
        return {"prompt": row["prompt"], "chosen": row["response_1"], "rejected": row["response_2"]}
    else:
        return {"prompt": row["prompt"], "chosen": row["response_2"], "rejected": row["response_1"]}

processed_data = df.apply(map_responses, axis=1).to_list()
dataset = Dataset.from_list(processed_data)

# Step 2: Initialize the tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 3: Tokenize the dataset
def tokenize_function(batch):
    # Tokenize "chosen" and "rejected" responses with the prompt
    tokenized_chosen = tokenizer(
        batch["prompt"], batch["chosen"], truncation=True, padding="max_length", max_length=128
    )
    tokenized_rejected = tokenizer(
        batch["prompt"], batch["rejected"], truncation=True, padding="max_length", max_length=128
    )

    # Return tokenized results
    return {
        "input_ids_chosen": tokenized_chosen["input_ids"],
        "attention_mask_chosen": tokenized_chosen["attention_mask"],
        "input_ids_rejected": tokenized_rejected["input_ids"],
        "attention_mask_rejected": tokenized_rejected["attention_mask"],
    }

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Step 4: Initialize the reward model
reward_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

# Step 5: Define RewardConfig
reward_config = RewardConfig(
    output_dir="./reward_model_output",  # Directory to save model
    max_length=128,  # Maximum sequence length
    per_device_train_batch_size=8,  # Training batch size
    per_device_eval_batch_size=8,  # Evaluation batch size
    learning_rate=5e-5,  # Learning rate
    num_train_epochs=3,  # Number of training epochs
    evaluation_strategy="steps",  # Evaluate periodically
    logging_strategy="steps",  # Logging configuration
    logging_steps=10,  # Log progress every 10 steps
    save_steps=500,  # Save model every 500 steps
    remove_unused_columns=False,  # Required for pretokenized datasets
    report_to="none",  # Report to TensorBoard
)

# Step 6: Define DataCollator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Step 7: Create RewardTrainer
trainer = RewardTrainer(
    model=reward_model,
    args=reward_config,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,  # Pass tokenizer for preprocessing
)

# Step 8: Train the reward model
trainer.train()

# Evaluate the reward model
results = trainer.evaluate()
print("Evaluation Results:", results)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-1d09f31db149>:83: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.15.0 for `RewardTrainer.__init__`. Use `processing_class` instead.
  trainer = RewardTrainer(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ [CLS] tell me a joke. [SEP] why did the       │ [CLS] tell me a joke. [SEP] knock, knock!    │ [0.6631, 0.3369] │
│ chicken cross the road? [SEP]                 │ [SEP]                                        │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ [CLS] what is ai? [SEP] a type of plant.      │ [CLS] what is ai? [SEP] artificial           │ [0.5313, 0.4687] │
│ [SEP]                                         │ intelligence. [SEP]                          │                  │
└───────────────────────────────────────────────┴──────────────────────────────────────────────┴──────────────────┘

Evaluation Results: {'eval_loss': 0.4893132150173187, 'eval_accuracy': 1.0, 'eval_runtime': 1.7721, 'eval_samples_per_second': 1.129, 'eval_steps_per_second': 0.564, 'epoch': 3.0}


In [ ]:
# Evaluate the reward model
results = trainer.evaluate()
print("Evaluation Results:", results)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ [CLS] tell me a joke. [SEP] why did the       │ [CLS] tell me a joke. [SEP] knock, knock!    │ [0.6746, 0.3254] │
│ chicken cross the road? [SEP]                 │ [SEP]                                        │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ [CLS] what is ai? [SEP] a type of plant.      │ [CLS] what is ai? [SEP] artificial           │ [0.5571, 0.4429] │
│ [SEP]                                         │ intelligence. [SEP]                          │                  │
└───────────────────────────────────────────────┴──────────────────────────────────────────────┴──────────────────┘

Evaluation Results: {'eval_loss': 0.4893132150173187, 'eval_accuracy': 1.0, 'eval_runtime': 3.1124, 'eval_samples_per_second': 0.643, 'eval_steps_per_second': 0.321, 'epoch': 3.0}


In [ ]:
output = trainer.predict(tokenized_dataset)
predictions = output.predictions
print(predictions)  # Output: model's predicted values/logits for the dataset

[[0.6745839  0.3254161 ]
 [0.55712414 0.44287583]]


In [ ]:
# Assuming `output.predictions` contains the predicted scores for chosen and rejected responses
score_chosen = predictions[:, 0]  # Predicted score for the chosen response
score_rejected = predictions[:, 1]  # Predicted score for the rejected response

# Compute the margin as the difference between chosen and rejected scores
margin = score_chosen - score_rejected

In [ ]:
print(margin)

[0.34916782 0.11424831]
